## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id') # 실제 시험장에서는 제공되지 않고, 자가 채점을 위해 불러옵니다.

In [3]:
# 반복문을 구성하여 처리해 봅니다.

# 처리 내용을 정의합니다, (대상 변수명, 치환할 내용, 치환후 수준별 카운트)
repl_list = [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]), 
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]),
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]

for c, d, cnt in repl_list:
    print(c, d, cnt)
    s_repl = df_train[c].replace(d) # 치환을 합니다. (아직 반영은 하지 않습니다.)
    if not (s_repl.value_counts().sort_index() == cnt).all(): # 치환후 카운트를 체크합니다.
        print("Error", c, d, cnt, s_repl.value_counts().sort_index()) # 에러 내용을 출력합니다.
        break
    df_train[c] = s_repl # 치환한 결과를 반영합니다.
    df_test[c] = df_test[c].replace(d) # 테스트에 대해서도 반영합니다.

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [4]:
# 문제 4번을 활용하기 위해 만듭니다.
df_train['targetA'] = df_train['target'] <= 7.45

In [5]:
q = [i for i in np.arange(0, 1.01, 0.01)]
# 나머지 변수에 대해서도 해당 파생 변수를 만들어 줍니다.
for i in range(0, 14):
    col = 'cont{}'.format(i)
    col_q = col + '_q'
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_range = pd.cut(df_train[col], q_val, right=True)
    q_mean = df_train.groupby(q_range)['target'].mean()
    df_train[col_q] = q_range.map(q_mean).astype('float')
    df_test[col_q] = pd.cut(df_test[col], q_val, right=True).map(q_mean).astype('float')

In [6]:
from scipy.stats import norm

mu_A, s_A = 6.769, 0.616
mu_B, s_B = 8.123, 0.527

df_train_clf = df_train.assign(
    # 귀무가설 : target은 A입니다, 대립가설: target은 B입니다.
    prob_A = 1 - norm.cdf(df_train['target'], loc=mu_A, scale=s_A),
    # 귀무가설 : target은 B입니다, 대립가설: target은 A입니다.
    prob_B = norm.cdf(df_train['target'], loc=mu_B, scale=s_B)
)
df_train_clf = df_train_clf.query('prob_B < 0.01 or prob_A < 0.01').copy()

In [7]:
# 공통
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit, KFold 

cv = KFold(n_splits=5, random_state=123)
# train(80%)/test(20%) 한 번으로 검증합니다. XGB, RF등 오래 걸리는 모델을 위해 사용합니다.
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=123)

df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
X = df_test.columns.tolist() + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']

cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]
cont_q_cols = ['cont{}_q'.format(i) for i in range(14)]

# 위에서 발생한 leak을 바로 잡아 교차검증을 합니다.
q = [i for i in np.arange(0, 1.01, 0.01)]
def eval_model(model, sp):
    score_train, score = list(), list()
    for train_idx, test_idx in sp.split(df_train):
        df_cv_train, df_cv_test = df_train.iloc[train_idx].copy(), df_train.iloc[test_idx].copy()
        # 검증셋에서 train으로 파생변수를 만들고
        # 검증셋의 test(겹외셋)에 검증셋의 train으로 만든 통계값(mean)을 반영합니다.
        for i in range(0, 14):
            col = 'cont{}'.format(i)
            col_q = col + '_q'
            q_val = df_cv_train[col].quantile(q)
            q_val.iloc[[0, -1]] = [-np.inf, np.inf]
            q_range = pd.cut(df_cv_train[col], q_val, right=True)
            q_mean = df_cv_train.groupby(q_range)['target'].mean()
            df_cv_train[col_q] = q_range.map(q_mean).astype('float')
            df_cv_test[col_q] = pd.cut(df_cv_test[col], q_val, right=True).map(q_mean).astype('float')
        model.fit(df_cv_train[X], df_cv_train['target'])
        score_train.append(-(mean_squared_error(df_cv_train['target'], model.predict(df_cv_train[X]))) ** 0.5)
        score.append(-(mean_squared_error(df_cv_test['target'], model.predict(df_cv_test[X]))) ** 0.5)
    return score_train, score

def choose_model(model):
    model.fit(df_train[X], df_train['target'])
    prd = model.predict(df_test[X])
    pd.DataFrame({
        'target': prd
    }, index=df_test.index).to_csv('answer6.csv')
    return prd 

In [8]:
# 문제 4번 모델을 만듭니다. targetA일 확률을 활용할 예정입니다.
import xgboost as xgb

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_xgb = ['cont{}'.format(i) for i in range(14)] + ['cat{}'.format(i) for i in range(10)]
clf_xgb = make_pipeline(
    ct,
    xgb.XGBClassifier(
        max_depth = 2, # 트리의 최대 깊이 2
        reg_alpha = 0.1, # L1 규제 0.1
        reg_lambda = 0.1, # L2 규제 0.1
        colsample_bytree=0.25, # 트리 당 컬럼 샘플링 비율 0.25
        n_estimators=500, # 트리의 수 500
        random_state=123, # random_state 123
    )
)

X_xgb = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
clf_xgb.fit(df_train_clf[X_xgb], df_train_clf['targetA'])
df_train['targetA_prob'] = clf_xgb.predict_proba(df_train[X_xgb])[:, 1]
df_test['targetA_prob'] = clf_xgb.predict_proba(df_test[X_xgb])[:, 1]

In [9]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_cols)
])
reg_lr = make_pipeline(
    ct, 
    LinearRegression()
)
scores_train, scores = eval_model(reg_lr, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8632456423386847, 0.0029474142614306633, -0.8630025788522048)

In [10]:
prd = choose_model(reg_lr)
# 자가 채점을 해봅니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8657267201878257

In [11]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_cols),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_2 = make_pipeline(
    ct, 
    LinearRegression()
)
scores_train, scores = eval_model(reg_lr_2, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8435148589687671, 0.003012400908502579, -0.8432679868397729)

In [12]:
prd = choose_model(reg_lr_2)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8493184210459026

In [13]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_q_cols),
])
reg_lr_3 = make_pipeline(
    ct, 
    LinearRegression()
)
scores_train, scores = eval_model(reg_lr_3, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8486955888442786, 0.0029782918718652706, -0.8446443969137316)

In [14]:
prd = choose_model(reg_lr_3)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8508248225223494

In [15]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_q_cols),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_4 = make_pipeline(
    ct, 
    LinearRegression()
)
scores_train, scores = eval_model(reg_lr_4, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8436136042735681, 0.002966661670507662, -0.8414487168951119)

In [16]:
prd = choose_model(reg_lr_4)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8479063767656508

In [17]:
from sklearn.decomposition import PCA

corr_cols = ['cont0', 'cont5', 'cont8', 'cont9', 'cont12']
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std_pca', make_pipeline(StandardScaler(), PCA(n_components=3)), corr_cols),
    ('std', StandardScaler(), [i for i in cont_cols if i not in corr_cols]),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_5 = make_pipeline(
    ct, 
    LinearRegression()
)
scores_train, scores = eval_model(reg_lr_2, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8435148589687671, 0.003012400908502579, -0.8432679868397729)

In [18]:
prd = choose_model(reg_lr_5)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8492943392490968

In [19]:
from sklearn.ensemble import RandomForestRegressor

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_cols),
    ('pt', 'passthrough', cont_cols + ['targetA_prob'])
])

reg_rf = make_pipeline(
    ct,
    RandomForestRegressor(n_estimators=50, max_depth=4, random_state=123, n_jobs=4)
)
scores_train, scores = eval_model(reg_rf, ss)
np.mean(scores), np.mean(scores_train)

(-0.8432410423704048, -0.8430963969319142)

In [20]:
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_cols),
    ('pt', 'passthrough', cont_cols + ['targetA_prob'])
])
reg_xgb = make_pipeline(
    ct,
    xgb.XGBRegressor(n_estimators=150, max_depth=2, random_state=123, n_jobs=4)
)
scores_train, scores = eval_model(reg_xgb, ss)
np.mean(scores), np.mean(scores_train)

(-0.8426761026271767, -0.8414261158818475)

In [21]:
from sklearn.ensemble import VotingRegressor

reg_vt = VotingRegressor([
    #('lr_1', reg_lr),
    #('lr_2', reg_lr_2),
    #('lr_3', reg_lr_3),
    ('lr_4', reg_lr_4),
    #('lr_5', reg_lr_5),
    ('rf', reg_rf),
    ('xgb', reg_xgb),
])
scores_train, scores = eval_model(reg_vt, ss)
np.mean(scores), np.mean(scores_train)

(-0.8426363022718273, -0.8414389546822001)

In [22]:
prd = choose_model(reg_vt)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.847813580463817